# Finding apparent horizons in volume data {#examples_find_horizons}

In this example we will generate some numeric volume data representing a Kerr
black hole and then find its apparent horizon.

In [ ]:
# Distributed under the MIT License.
# See LICENSE.txt for details.

# Dependencies:
%pip install numpy matplotlib pandas 'h5py>=3.0.0' ruamel.yaml

In [56]:
import h5py
import matplotlib.pyplot as plt
import os
import pandas as pd
from pathlib import Path
from copy import deepcopy
from ruamel.yaml import YAML

In [57]:
%%bash
# Clean up output files from previous runs
rm -f Kerr*.h5
rm -f FindHorizons*.h5

First, make sure you have compiled the `SolveXcts` and `FindHorizons3D`
executables in `Release` mode. Put the path to your build directory below.

In [58]:
SPECTRE_BUILD_DIR = Path("/home/himanshu/Desktop/spectre/build-Shared_libs-Release")
SPECTRE_HOME = Path("/home/himanshu/Desktop/spectre")

## Generate Kerr volume data

We run the `SolveXcts` executable to generate Kerr volume data. We could also
just invoke the Kerr analytic solution and write the data to disk, but the
`SolveXcts` executable already writes volume data in the correct format.

In [59]:
# Load example input file
kerrschild_input_file_path = os.path.join(
    SPECTRE_HOME, "tests/InputFiles/Xcts/KerrSchild.yaml"
)
yaml = YAML()
with open(kerrschild_input_file_path, "r") as open_input_file:
    kerr_input_file = list(yaml.load_all(open_input_file))

    # Modify Kerr-Schild example input file
    # - Set some interesting Kerr parameters
    kerr_input_file[1]["Background"]["KerrSchild"] = dict(
        Mass=1.0, Spin=[0.0, 0.0, 0.0], Center=[0.0, 0.0, 0.0]
    )
    # - Set the initial guess to the solution to converge quickly
    kerr_input_file[1]["InitialGuess"] = kerr_input_file[1]["Background"]
    # - Choose domain parameters
    domain_params = kerr_input_file[1]["DomainCreator"]["Sphere"]
    domain_params["InnerRadius"] = 1.0
    domain_params["InitialRefinement"] = 0
    domain_params["InitialGridPoints"] = [8, 8, 8]

    # Change boundary conditions to negative expansion boundary condition
    IBC = domain_params["Interior"]["ExciseWithBoundaryCondition"]
    del IBC["AnalyticSolution"]
    IBC["ApparentHorizon"] = {
      "Center": [0. ,0. ,0.],
      "Rotation": [0., 0., 0.],
      "Lapse": kerr_input_file[1]["Background"],
      "NegativeExpansion": kerr_input_file[1]["Background"]
    }

    # - Allow the elliptic solver to converge
    kerr_input_file[1]["NonlinearSolver"]["NewtonRaphson"]["ConvergenceCriteria"][
        "MaxIterations"
    ] = 100
    # - Set output file names
    kerr_input_file[1]["Observers"]["VolumeFileName"] = "KerrVolume"
    kerr_input_file[1]["Observers"]["ReductionFileName"] = "KerrReductions"

# Write modified input file
with open("Kerr.yaml", "w") as open_input_file:
    yaml.dump_all(kerr_input_file, open_input_file)

In [60]:
SOLVE_XCTS = os.path.join(SPECTRE_BUILD_DIR, "bin/SolveXcts")
!{SOLVE_XCTS} --input-file Kerr.yaml +auto-provision

Charm++: standalone mode (not using charmrun)
Charm++> Running in Multicore mode: 20 threads (PEs)
Converse/Charm++ Commit ID: v7.0.0
CharmLB> Load balancer assumes all CPUs are same.
Charm++> Running on 1 hosts (1 sockets x 10 cores x 2 PUs = 20-way SMP)
Charm++> cpu topology info is gathered in 0.002 seconds.

Executing 'SolveXcts' using 20 processors.
Launch command line: /home/himanshu/Desktop/spectre/build-Shared_libs-Release/bin/SolveXcts --input-file Kerr.yaml
Charm++ startup time in seconds: 0.536004
Date and time at startup: Fri Nov 10 00:17:08 2023

SpECTRE Build Information:
Version:                      2023.10.11
Compiled on host:             0e335e144dbb
Compiled in directory:        /home/himanshu/Desktop/spectre/build-Shared_libs-Release
Source directory is:          /home/himanshu/Desktop/spectre
Compiled on git branch:       develop
Compiled on git revision:     006ed0ff93
Linked on:                    Thu Oct  5 01:54:50 2023
Build type:                   Release

Th

## Find horizons in the volume data

We run the `FindHorizons3D` executable over the generated volume data:

In [61]:
# Load example input file
horizons_input_file_path = os.path.join(
    SPECTRE_HOME, "tests/InputFiles/FindHorizons/FindHorizons3D.yaml"
)
with open(horizons_input_file_path, "r") as open_input_file:
    horizons_input_file = list(yaml.load_all(open_input_file))

    # Modify Kerr-Schild example input file
    # - Select the same domain that we solved above
    horizons_input_file[1]["DomainCreator"] = deepcopy(
        kerr_input_file[1]["DomainCreator"]
    )
    del horizons_input_file[1]["DomainCreator"]["Sphere"]["OuterBoundaryCondition"]
    horizons_input_file[1]["DomainCreator"]["Sphere"]["Interior"] = "Excise"
    # - Set importer file names
    importer_params = horizons_input_file[1]["Importers"]["VolumeData"]
    importer_params["FileGlob"] = "KerrVolume*.h5"
    importer_params["ObservationValue"] = "Last"
    # - AH finder parameters
    ah_params = horizons_input_file[1]["ApparentHorizons"]["AhA"]
    ah_params["InitialGuess"]["Radius"] = 1.0
    # - Set output file names
    horizons_input_file[1]["Observers"]["VolumeFileName"] = "FindHorizonsVolume"
    horizons_input_file[1]["Observers"]["ReductionFileName"] = "FindHorizonsReductions"
    horizons_input_file[1]["Observers"]["SurfaceFileName"] = "FindHorizonsSurfaces"

    # Write modified input file
    with open("FindHorizons.yaml", "w") as open_input_file:
        yaml.dump_all(horizons_input_file, open_input_file)

In [62]:
FIND_HORIZONS = os.path.join(SPECTRE_BUILD_DIR, "bin/FindHorizons3D")
!{FIND_HORIZONS} --input-file FindHorizons.yaml +auto-provision

Charm++: standalone mode (not using charmrun)
Charm++> Running in Multicore mode: 20 threads (PEs)
Converse/Charm++ Commit ID: v7.0.0
CharmLB> Load balancer assumes all CPUs are same.
Charm++> Running on 1 hosts (1 sockets x 10 cores x 2 PUs = 20-way SMP)
Charm++> cpu topology info is gathered in 0.000 seconds.

Executing 'FindHorizons3D' using 20 processors.
Launch command line: /home/himanshu/Desktop/spectre/build-Shared_libs-Release/bin/FindHorizons3D --input-file FindHorizons.yaml
Charm++ startup time in seconds: 0.021654
Date and time at startup: Fri Nov 10 00:17:17 2023

SpECTRE Build Information:
Version:                      2023.10.11
Compiled on host:             0e335e144dbb
Compiled in directory:        /home/himanshu/Desktop/spectre/build-Shared_libs-Release
Source directory is:          /home/himanshu/Desktop/spectre
Compiled on git branch:       develop
Compiled on git revision:     006ed0ff93
Linked on:                    Thu Oct  5 01:54:50 2023
Build type:            

## Horizon quantities

The horizon finder measures a few surface quantities:

In [63]:
# These routines read the data and process them a bit. You can skip them to see
# the results below.


def load_dataset(subfile):
    legend = subfile.attrs["Legend"]
    return pd.DataFrame(data=subfile, columns=legend).set_index(legend[0])

In [64]:
with h5py.File("FindHorizonsReductions.h5", "r") as open_h5_file:
    ah = load_dataset(open_h5_file["AhA.dat"])

In [65]:
print(ah.to_string())

           Area  IrreducibleMass  MaxRicciScalar  MinRicciScalar  ChristodoulouMass  DimensionlessSpinMagnitude
Time                                                                                                           
0.0   50.267921         1.000024        0.500223        0.499537           1.000024                3.565439e-16
